<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# PA005: High Value Customer Identification ( Insiders )

# 0.0 Imports

In [1]:
import re
import sqlite3

import pandas                                                                             as pd
import numpy                                                                              as np
import seaborn                                                                            as sns
import umap.umap_                                                                         as umap


from sqlalchemy                    import create_engine
from datetime                      import datetime

from matplotlib                    import pyplot                                          as plt

from sklearn                       import cluster                                         as c
from sklearn                       import metrics                                         as met
from sklearn                       import decomposition                                   as dd
from sklearn                       import ensemble                                        as en
from sklearn                       import mixture                                         as mx
from plotly                        import express                                         as px
from sklearn                       import preprocessing                                   as pp


import warnings
warnings.filterwarnings("ignore")

##  0.2 Load Dataset


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
echo %cd%


SyntaxError: invalid syntax (<ipython-input-2-10aedb44ba15>, line 1)

In [ ]:
# load data
path = '/Users/felip/repositorio/pa005_insiders_clustering/insiders_clustering/'
df_raw = pd.read_csv(path + 'data/raw/Ecommerce.csv', encoding='cp1252')

# # drop extra column
df_raw = df_raw.drop(columns=['Unnamed: 8'] , axis=1)


# <font color ='red'> 1.0 Descrição dos dados </font>

In [ ]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [ ]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date','unit_price', 'customer_id', 'country']

df1.columns = cols_new

## 1.2 Data dimensios

In [ ]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

## 1.3 Data types

In [ ]:
df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum()

## 1.5 Replace NA


In [ ]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]



In [ ]:
# create referance 

df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000 , 19000+len(df_backup), 1)
df_backup.head()

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup , on='invoice_no' , how='left')

#coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns

df1 = df1.drop(columns=['customer_id_x' , 'customer_id_y' ] , axis = 1)

df1.head()

## 1.6 Changes dtypes


In [ ]:
# invoice date

df1['invoice_date'] = pd.to_datetime(df1['invoice_date'] , format='%d-%b-%y')

# customer_id

df1['customer_id'] = df1['customer_id'].astype('int64')


## 1.7 Descriptive Statistics

In [ ]:
num_attributes = df1.select_dtypes(include=['int64' , 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64' , 'float64','datetime64[ns]'])


### 1.7.1 Numerical Attributs

In [ ]:
# central tendency - mean, median
mean = pd.DataFrame(num_attributes.apply(np.mean)).T
median = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - std , min , max , range , skew , kurtosis
std = pd.DataFrame(num_attributes.apply(np.std)).T
mi = pd.DataFrame(num_attributes.apply(np.min)).T
ma = pd.DataFrame(num_attributes.apply(np.max)).T
ran = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
kurtosis = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate

m = pd.concat([mi , ma ,ran, mean , median  , std,  skew ,  kurtosis]).T.reset_index()
m.columns = ['Attributes' , 'min' , 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtoses']
m

### 1.7.2 Categorical Attributs

#### Invoice No

In [ ]:
# identificação:
df_letter_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))) , :]
df_letter_invoices.head()

print('Total number of invoices: {}'.format(len(df_letter_invoices)))
print('Total number of negative quantity: {}'.format(len(df_letter_invoices['quantity'] < 0)))

#### Stock Code

In [ ]:
# check stock code only characters
df1.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))),'stock_code'].unique()

# Ação:
## 1. Remove stock_code in ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK']


# <font color ='red'> 2.0 Filtragem de variaveis </font>

In [ ]:
df2 = df1.copy()

In [ ]:
# ==== Numerical Attributes ====

df2 = df2.loc[df2['unit_price'] > 0.04 , :]


# ==== Categorical Attributes ====


df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop('description', axis =1)

# map
df2 = df2[~df2['country'].isin (['European Community' , 'Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin( [16446]) ]


## quantity 
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] >= 0, :]


# <font color ='red'> 3.0 Feature Engeneering </font>

In [ ]:
df3 = df2.copy()

## 3.1 Feature Creation

In [ ]:
# data reference

df_ref = df3.drop(['invoice_no','stock_code','quantity', 'invoice_date','unit_price','country'], axis = 1 ).drop_duplicates(ignore_index = True)


### 3.1.1. Gross Revenue

In [ ]:
# Gross Revenue ( Faturamento = quantity * price)
df2_purchase.loc[: , 'gross_revenue'] = df2_purchase.loc[: , 'quantity'] * df2_purchase.loc[: , 'unit_price']

df_monetary = df2_purchase.loc[: , ['customer_id' , 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary,on='customer_id',how='left')

df_ref.isna().sum()

### 3.1.2. Recency - Day from last purchase

In [ ]:
# Recency - Last day purchase
df_recency = df2_purchase.loc[: ,[ 'customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id','recency_days']].copy()
df_ref = pd.merge(df_ref , df_recency , on='customer_id', how='left')
df_ref.isna().sum()

### 3.1.3. Quantity of purchased

In [ ]:
# Numero de produtos
df_freq = (df2_purchase.loc[: ,['customer_id','invoice_no']].drop_duplicates()
                                                            .groupby('customer_id')
                                                            .count()
                                                            .reset_index()
                                                            .rename(columns={'invoice_no' : 'qtde_invoices'}))

df_ref = pd.merge(df_ref , df_freq , on='customer_id', how='left')
df_ref.isna().sum()

### 3.1.4. Quantity total of items purchased


In [ ]:
df_freq = df2_purchase.loc[: ,['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns = {'quantity' : 'qtde_items'})
df_ref = pd.merge(df_ref , df_freq , on='customer_id', how='left')
df_ref.isna().sum()

### 3.1.5. Quantity of products purchased


In [ ]:
df_freq = df2_purchase.loc[: ,['customer_id','stock_code']].groupby('customer_id').count().reset_index().rename(columns = {'stock_code' : 'qtde_products'})
df_ref = pd.merge(df_ref , df_freq , on='customer_id', how='left')
df_ref.isna().sum()

### 3.1.6. Avg Ticket Value

In [ ]:
# Avg ticket
df_avg_ticket = df2_purchase.loc[: ,['customer_id' , 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue':'avg_ticket'})
#df_avg_ticket['avg_ticket'] = np.round(df_avg_ticket['avg_ticket'] , 2)
df_ref = pd.merge(df_ref, df_avg_ticket,on='customer_id',how='left')
df_ref.isna().sum()

### 3.1.8. Frequency Purchase

In [ ]:
df3_purchase = df2_purchase.copy()

In [ ]:
df_aux = df3_purchase[['invoice_no','customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').agg(max_  = ('invoice_date', 'max'), 
                                                                                               min_  = ('invoice_date', 'min'), 
                                                                                               days_ = ('invoice_date', lambda x: (( x.max() - x.min() ).days)+ 1),
                                                                                               buy_  = ('invoice_date', 'count')).reset_index()

# Frequency 
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1)

# Merge
df_ref = pd.merge(df_ref , df_aux[['customer_id', 'frequency']], on='customer_id', how = 'left')

df_ref.isna().sum()

### 3.1.9. Numbers of Returns

In [ ]:
df_returns = df2_returns[['customer_id' , 'quantity']].groupby('customer_id').sum().reset_index().rename( columns={'quantity' : 'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1
df_ref = pd.merge(df_ref , df_returns, on ='customer_id' ,  how='left')

df_ref.loc[df_ref['qtde_returns'].isna() , 'qtde_returns'] = 0
df_ref.isna().sum()

# <font color ='red'> 4.0 EDA (Exploratory Data Analysis) </font>

In [ ]:
df4 = df_ref.dropna().copy()

## 4.3 Estudo do espaço

In [ ]:
# Selected Dataset
cols_selected = ['customer_id', 'gross_revenue','recency_days', 'qtde_products', 'frequency', 'qtde_returns' ]
df43 = df4[cols_selected].drop(columns = 'customer_id')


In [ ]:
mm = pp.MinMaxScaler()

df43['gross_revenue']          = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days']           = mm.fit_transform(df43[['recency_days']])
df43['qtde_products']          = mm.fit_transform(df43[['qtde_products']])
df43['frequency']              = mm.fit_transform(df43[['frequency']])
df43['qtde_returns']           = mm.fit_transform(df43[['qtde_returns']])


X = df43.copy()


In [ ]:
# training dataset 
X = df43.drop(columns=[ 'gross_revenue'], axis = 1)
Y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators = 300 , random_state = 42 )

# model training
rf_model.fit(X,Y)

# dataframe Leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

In [ ]:
# tree

num_comp=[2,3,4,5]

components_list1=[]
components_list2=[]
components_list3=[]
components_list4=[]

df_tree_1 = pd.DataFrame()
df_tree_2 = pd.DataFrame()
df_tree_3 = pd.DataFrame()
df_tree_4 = pd.DataFrame()



for i in num_comp:
    
    # reduzer dimensionality
    reducer = umap.UMAP(random_state = 42, n_components = i)
    embedding = reducer.fit_transform(df_leaf)

    
    if i == 2:
        components_list1.append(embedding)
        # embedding
        df_tree_1['embedding_x'] = embedding[:,0]
        df_tree_1['embedding_y'] = embedding[:,1]


        
    if i == 3:
        components_list2.append(embedding)
        # embedding
        df_tree_2['embedding_x'] = embedding[:,0]
        df_tree_2['embedding_y'] = embedding[:,1]
        df_tree_2['embedding_z'] = embedding[:,2]

        
    if i == 4:
        components_list3.append(embedding)
        df_tree_3['embedding_x'] = embedding[:,0]
        df_tree_3['embedding_y'] = embedding[:,1]
        df_tree_3['embedding_z'] = embedding[:,2]
        df_tree_3['embedding_w'] = embedding[:,3]

    if i == 5:
        components_list4.append(embedding)
        df_tree_4['embedding_x'] = embedding[:,0]
        df_tree_4['embedding_y'] = embedding[:,1]
        df_tree_4['embedding_z'] = embedding[:,2]
        df_tree_4['embedding_w'] = embedding[:,3]
        df_tree_4['embedding_l'] = embedding[:,4]


# <font color ='red'> 5.0 Data Preparation </font>

In [ ]:
# UMAP EMbedding
df5 = df_tree_2.copy()
df5.to_csv('../src/data/tree_based_embedding.csv' , index = False)




# <font color ='red'> 5.0 Hyperparameter Fine-Tunning </font>

In [ ]:
x = df5.copy()

In [ ]:
clusters = np.arange(2,25,1)

In [ ]:
kmeans_list = []

for k in clusters:

    # model definition
    kmeans_model = c.KMeans(n_clusters = k , random_state=42)

    # model training
    kmeans_model.fit(x)

    # model predict
    labels = kmeans_model.predict(x)

    # model performance
    sil = met.silhouette_score(x , labels , metric='euclidean')

    kmeans_list.append(sil)

# <font color ='red'> 6.0 Model Training </font>

In [ ]:
df8 = x.copy()

## 6.1 GMM

In [ ]:
# model definition
k = 11
kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)

# model training
kmeans.fit(df8)

# clustering
labels = kmeans.labels_



## 6.2 Cluster Validation

In [ ]:
print('SS value: {}'.format(met.silhouette_score(df8, labels, metric='euclidean')))

# <font color ='red'> 7.0 Cluster Analysis </font>

In [ ]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels



df92['last_training_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# change dtypes
df92['recency_days'] = df92['recency_days'].astype('int64')
df92['qtde_products'] = df92['qtde_products'].astype('int64')
df92['qtde_returns'] = df92['qtde_returns'].astype('int64')
df92['last_training_timestamp'] =pd.to_datetime(df92['last_training_timestamp'])

## 7.2 Cluster Profile


In [ ]:
# Number of customer
df_cluster = df92[['customer_id','cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customers'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg Gross revenue
df_avg_gross_revenue = df92[['gross_revenue','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_avg_gross_revenue , how='inner' , on='cluster')

# Avg recency days
df_avg_recency_days = df92[['recency_days','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_avg_recency_days , how='inner' , on='cluster')

# Avg invoice_no
df_qtde_products = df92[['qtde_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_qtde_products , how='inner' , on='cluster')

# Frequency
df_frequency = df92[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_frequency , how='inner' , on='cluster')

# Returns
df_qtde_returns = df92[['qtde_returns','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_qtde_returns , how='inner' , on='cluster')


df_cluster

In [ ]:
# 1 cluster Insiders
# 4 cluster more products
# 7 cluster spend money
# 9 cluster even more products
# 2 cluster less days
# 10 cluster less 1k
# 8 cluster Stop Returners
# 9 cluster more buy
# 5 cluster even more buy
# 3 cluster weak
# 0 cluster latecomer

### Cluster 01: ( Candidato a Insiders)

    - Número de customers: 665 (12% do customers)
    - Faturamento médio: 7503
    - Recência média: 52 dias 
    - Média de produtos comprados: 371 produtos
    - Frequência de produtos comprados: 0.23 produtos/dia
    - Receita em média: $ 7503.14,00 dólares
    


# <font color ='red'> 8.0 Deploy to production </font>

In [ ]:
df92.dtypes

## 8.1. Insert into SQLITE

In [ ]:
# database connection
conn = create_engine('sqlite:///insiders_db.sqlite')

# drop table
# query_drop_insiders="""
#     DROP TABLE Insiders
# """


# create Table
# query_create_table_insiders = """
#     CREATE TABLE Insiders (
#         customer_id                   INTEGER,
#         gross_revenue                 REAL,
#         recency_days                  INTEGER,
#         qtde_products                 INTEGER,
#         frequency                     REAL,
#         qtde_returns                  INTEGER,
#         cluster                       INTEGER,
#         last_training_timestamp       TIMESTAMP
#      )

# """

# conn.execute(query_create_table_insiders)

In [ ]:
# insert data into
df92.to_sql('Insiders', con=conn, if_exists='append' , index=False)

In [ ]:
# get query
# query_collect="""
#     SELECT * from Insiders
# """

# df = pd.read_sql_query(query_collect , conn)
# df.head()